---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **다양한 `LLM` 모델 활용**

* 모델 혹은 **`LLM`(Large Language Model)** 단계는 **이전 프롬프트 단계에서 구성된 입력을 기반으로 대규모 언어 모델을 활용하여 응답을 생성하는 과정**

* 이 단계는 RAG 시스템의 핵심적인 부분

* **언어 모델의 능력을 최대한 활용하여 사용자의 질문에 대해 정확하고 자연스러운 답변을 생성**

### **`LLM`** 필요성

* **사용자 의도 이해** 
  * `LLM`은 `다양한 언어`의 `구조와 의미`를 깊이 이해하고 있으며, 이를 바탕으로 `복잡한 질문`에 `답`할 수 있음
  * **`자연어 이해(NLU)`** 와 **`자연어 생성(NLG)`** 능력이 결합되어, 보다 `자연스럽고 유익한 응답` 제공

* **문맥적 적응성**
  * `LLM`은 주어진 `문맥`을 `고려`하여 응답 생성 → 사용자의 질문에 더욱 정확하게 대응 가능
  * `사전학습`된 지식`외` + **사용자가 제공한 정보에 기반한 답변** 을 `문맥을 참고`하여 `답변`

### **`LLM`** 중요성

* `LLM 단계` = 사용자의 질문에 대한 **`답변의 질`** + **`자연스러움`** 을 결정짓는 **핵심 요소**

* `LLM` = `지금까지의 모든 데이터와 정보를 종합`하여 **사용자의 질문에 최적화된 답변을 생성**

* `LLM의 성능` → **`RAG` 시스템의 전체적인 성능과 사용자 만족도에 직접적으로 영향** 을 미침 → `RAG` 시스템을 사용하는 많은 응용 분야에서 매우 중요한 역할

In [1]:
# 환경변수 처리 및 클라어트 생성
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json

# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [ ]:
# LangSmith 추적 설정하기 (https:smith.langchin.com)
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable                                                             # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음"      # API 키 값은 직접 출력하지 않음
org = "설정됨" if os.getenv('LANGCHAIN_ORGANIZATION') else "설정되지 않음"                     # 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")


<small>

* 셀 출력

    ```markdown
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

---

## **`OpenAI`**

### **개요**

- `OpenAI`는 **채팅 전용** `Large Language Model (LLM)`을 제공
- 이 모델을 생성할 때 `다양한 옵션`을 `지정`할 수 있으며, 이러한 옵션들은 모델의 동작 방식에 영향을 미침

### **옵션 상세 설명**

- `temperature`

  - 샘플링 온도를 설정하는 옵션
  - 값은 `0과 2 사이`에서 선택 가능
    - `높은 값(예: 0.8)` = **출력을 더 무작위하게 만듦 ≒ 높은 창의성** 
    - `낮은 값(예: 0.2)` = 출력을 더 집중되고 결정론적으로 만듦 ≒ **낮은 창의성**

- `max_tokens`

  - 채팅 완성에서 **`생성할 토큰의 최대 개수` 지정**
  - 모델이 **한 번에 생성할 수 있는 텍스트의 길이를 제어**

- `model_name`

  - 적용 가능한 모델을 선택하는 옵션
  - 더 자세한 정보는 [OpenAI 모델 문서](https://platform.openai.com/docs/models)에서 확인

**모델 스펙**

- [링크](https://platform.openai.com/docs/models): https://platform.openai.com/docs/models

| Model               | Input (1M) | Cached Input (1M) | Output (1M) | Context Window | Max Output Tokens | Knowledge Cutoff |
|---------------------|------------|-------------------|-------------|----------------|-------------------|------------------|
| gpt-4.1             | $2.00      | $0.50             | $8.00       | 1,047,576      | 32,768            | Jun 01, 2024     |
| gpt-4.1-mini        | $0.40      | $0.10             | $1.60       | 1,047,576      | 32,768            | Jun 01, 2024     |
| gpt-4.1-nano        | $0.10      | $0.025            | $0.40       | 1,047,576      | 32,768            | Jun 01, 2024     |
| gpt-4o              | $2.50      | $1.25             | $10.00      | 128,000        | 16,384            | Oct 01, 2023     |
| gpt-4o-mini         | $0.15      | $0.075            | $0.60       | 128,000        | 16,384            | Oct 01, 2023     |
| o1                  | $15.00     | $7.50             | $60.00      | 128,000        | 65,536            | Oct 01, 2023     |
| o1-mini             | $1.10      | $0.55             | $4.40       | 128,000        | 65,536            | Oct 01, 2023     |
| o1-pro              | $150.00    | –                 | $600.00     | 128,000        | 65,536            | Oct 01, 2023     |
| o3-mini             | $1.10      | $0.55             | $4.40       | 200,000        | 100,000           | Oct 01, 2023     |
| gpt-4.5-preview     | $75.00     | $37.50            | $150.00     | –              | –                 | –                |

In [ ]:
# OpenAI 모델 호출해보기

import openai
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수에서 OpenAI API 키 읽기
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI API 키 설정
openai.api_key = api_key

# 호출 확인 (Chat API 호출)
response = openai.completions.create(
    model="gpt-4o-mini",                            # gpt-4o-mini 모델 사용
    prompt="Hello, world!",
    max_tokens=5
)

print(response.choices[0].text.strip())             # This is a simple HTML (1.1s)

<small>

*  셀 출력 (조건 = max_tokens=5) (1.1s)

    ```markdown
    This is a simple HTML
    ```

* 모델 답변 이유
  
  * **모델의 기본적인 패턴 인식**
    * `OpenAI` 모델은 많은 데이터를 학습하면서 **`Hello, world!`**라는 문구가 종종 `프로그래밍` 관련 예시로 사용된다는 사실을 인식
    * **특히 이 문장은 프로그래밍 언어에서 자주 보이는 첫 번째 예제**

    * 예를 들어, `Python`, `JavaScript`, `HTML` 등에서 **`Hello, World!`** 를 출력하는 간단한 코드 많음
    * 그래서 모델은 그 문장을 보고 `이건 HTML 코드의 예시일 가능성이 크다`라고 추론해서 **`This is a simple HTML`** 을 출력한 것으로 보임
  
  * **모델의 자연어 처리 방식**
    * 모델은 주어진 텍스트의 맥락에 맞는 답을 생성하는데, **`Hello, world!`** 문장은 프로그래밍 예제에서 자주 사용 → **`간단한 HTML 코드`** 를 연상
    * 모델은 특정 문장을 **`이런 유형의 응답이 일반적으로 나오는 경우가 많다`** 고 학습해왔기 때문에 그런 출력을 낸 것

In [ ]:
# 다른 물음 던져보기
response = openai.completions.create(
    model="gpt-4o-mini",                            # gpt-4o-mini 모델 사용
    prompt="Say hello to the world in a friendly way.",
    max_tokens=50
)

print(response.choices[0].text.strip())             

<small>

* 셀 출력 (프롬프트 변경, max_tokens=50)(1.7s)

    ```markdown

    I'm your assistant, available anytime you need me. My purpose is to support and assist you with any questions or tasks you have. How can I help you today? 😊"
    ```
    ```markdown
    Hello! I'm your friendly assistant, here to help you anytime.
    ```


In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 객체 생성
gpt = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",                                   # 모델명
)

# 스트리밍 출력을 위하여 invoke() 대신 stream()을 사용
answer = gpt.stream("사랑이 뭔가요?")

# 답변 출력
#stream_response(answer)
for chunk in answer:
    print(chunk.content, end="", flush=True)

<small>

* 셀 출력 (temperature=0)(3.1s)

    ```plaintext
    사랑은 복잡하고 다면적인 감정으로, 사람들 간의 깊은 유대감이나 애정, 헌신을 포함합니다. 사랑은 가족, 친구, 연인 등 다양한 관계에서 나타날 수 있으며, 각기 다른 형태와 표현을 가집니다. 

    사랑은 종종 기쁨, 행복, 안정감을 주지만, 때로는 고통이나 상실감과 같은 어려운 감정을 동반하기도 합니다. 사랑은 서로를 이해하고 지지하며, 함께 성장하는 과정에서 중요한 역할을 합니다. 

    철학적, 심리학적, 문화적 관점에서 사랑은 다양한 해석이 가능하며, 각 개인의 경험에 따라 그 의미가 달라질 수 있습니다.
    ```

  ***

* 교재 속 코드(이전 방식) → 최신 방식으로 교체
  * 교재 속 코드 
  
  ```python
    stream_response(answer)
  ```

  * 최신 방식 
  ```python
    for chunk in answer:
        print(chunk.content, end="", flush=True)
  ```

---

## **`Anthropic`**

* `Anthropic`은 인공지능(AI) `안전성`과 `연구`에 중점을 둔 미국의 스타트업 기업

  * **설립 연도**: 2021년
  * **위치**: 미국 샌프란시스코
  * **창립자**: `OpenAI 출신 직원들` (Daniela Amodei와 Dario Amodei 등)
  * **기업 형태**: `공익기업` (`Public Benefit Corporation`)으로 등록

### **`Claude`**

* Claude는 Anthropic의 대표적인 대규모 언어 모델(LLM) 제품군입니다. 

  * **API 키 발급**: [https://console.anthropic.com/settings/keys](https://console.anthropic.com/settings/keys)
  * **모델 리스트**: [https://docs.anthropic.com/en/docs/about-claude/models](https://docs.anthropic.com/en/docs/about-claude/models)


![Claude](images/anthropic-20241023.png)

**업데이트 된 모델 스펙**

* 현재 앤트로픽의 모델은 **Claude 4**로 최신 버전 출시
* 제공된 모델 정보는 업데이트 필요 → 수정한 표는 아래와 같음


| 모델 이름               | 모델              | Anthropic API   | AWS Bedrock                    | GCP Vertex AI           |
| ------------------- | --------------- | --------------- | ------------------------------ | ----------------------- |
| **Claude 4**        | claude-4        | claude-4        | anthropic.claude-4-v1:0        | claude-4\@latest        |
| **Claude 4 Haiku**  | claude-4-haiku  | claude-4-haiku  | anthropic.claude-4-haiku-v1:0  | claude-4-haiku\@latest  |
| **Claude 4 Sonnet** | claude-4-sonnet | claude-4-sonnet | anthropic.claude-4-sonnet-v1:0 | claude-4-sonnet\@latest |
| **Claude 4 Opus**   | claude-4-opus   | claude-4-opus   | anthropic.claude-4-opus-v1:0   | claude-4-opus\@latest   |

* **주요 변경 사항**:

  * **Claude 4 모델**: 최신 모델로, `Claude 4`, `Claude 4 Haiku`, `Claude 4 Sonnet`, `Claude 4 Opus` 포함
  * **API 명**: 최신 API 명명 규칙에 맞게 `claude-4`, `claude-4-haiku`, `claude-4-sonnet`, `claude-4-opus`로 통합
  * **AWS Bedrock**와 **GCP Vertex AI**는 여전히 최신 버전으로 업데이트되어 있으며, **@latest**나 **-v1:0** 같은 태그가 사용

* **참고**:

  * `Claude 3.x` 시리즈는 여전히 일부 서비스에 사용될 수 있지만, 이제 대부분 **Claude 4** 모델로 업그레이드됨
  * `연말 출시 예정`으로 되어 있던 **Claude 3.5** 모델들은 이제 실제로 **Claude 4** 모델로 출시되었으므로, 사용 시 최신 모델을 확인하는 것이 중요

***

* **`Anthropic` 무료 API 리미트**

  * 모델별 사용 리미트 (1회당)

| 모델                | 요청 횟수 (분당) | 입력 토큰 (분당)       | 출력 토큰 (분당)      |
|---------------------|-----------------|------------------------|----------------------|
| **Claude Opus 4.x**  | 5회             | 최대 10,000 토큰 (캐시 읽기 제외) | 최대 4,000 토큰       |
| **Claude Sonnet 4**  | 5회             | 최대 10,000 토큰 (캐시 읽기 제외) | 최대 4,000 토큰       |
| **Claude Sonnet 3.7**| 5회             | 최대 10,000 토큰 (캐시 읽기 제외) | 최대 4,000 토큰       |
| **Claude Haiku 3.5** | 5회             | 최대 25,000 토큰 (캐시 읽기 제외) | 최대 5,000 토큰       |
| **Claude Haiku 3**   | 5회             | 최대 25,000 토큰 (캐시 읽기 제외) | 최대 5,000 토큰       |

  * 기타 리미트

| 항목                  | 리미트 값             |
|-----------------------|-----------------------|
| **배치 요청**          | 5회/분 (모든 모델 합산) |
| **웹 검색 도구 사용**  | 20회/초 (모든 모델 합산) |
| **파일 API 저장소**    | 100GB (조직 전체)     |

  * 하루에 사용할 수 있는 횟수

    * 각 모델은 **5회/분** 요청할 수 있으며, 하루 동안 최대 **7,200회** 요청할 수 있습니다.
    * 계산: **5회/분 × 60분 × 24시간 = 7,200회/일**


In [ ]:
# Anthropic 모델 호출해보기

import anthropic
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수에서 Anthropic API 키 읽기
api_key = os.getenv("ANTHROPIC_API_KEY")

print("\n--- ANTHROPIC API KEY 환경 변수 확인 ---")

if api_key:
    print(f"✅ api_key: {api_key}")
    print(" 앤트로픽 API 키가 환경변수로 잘 설정되었습니다.")
else:
    print("❌ 앤트로픽 API 키가 환경변수로 제대로 설정되지 않았습니다.")
    if not api_key:
        print(" ANTHROPIC_API_KEY 환경변수가 설정되지 않았거나, 비어 있습니다.")

<small>

* 셀 출력

    ```markdown
    --- ANTHROPIC API KEY 환경 변수 확인 ---
    ✅ api_key: sk-****
    앤트로픽 API 키가 환경변수로 잘 설정되었습니다.
    ```

In [ ]:
# Anthropic API 클라이언트 생성
client = None
try:
    client = anthropic.Anthropic(api_key=api_key)
    print("\n--- 클라이언트 생성 확인 ---")
    if client:
        print("✅ 클라이언트가 성공적으로 생성되었습니다.")
    else:
        print("❌ 클라이언트 생성에 실패했습니다.")
except Exception as e:
    print(f"❌ 클라이언트 생성 중 오류 발생: {e}")

<small>

* 셀 출력

    ```markdown
    --- 클라이언트 생성 확인 ---
    ✅ 클라이언트가 성공적으로 생성되었습니다.
    ```

In [ ]:
import anthropic

# Anthropic 클라이언트 생성
client = anthropic.Anthropic()

# 클라이언트 사용하여 메시지 스트림 생성
with client.messages.stream(
    max_tokens=50,  # 최대 토큰 수
    messages=[{"role": "user", "content": "Say hello to the world in a friendly way."}],  # 사용자 입력
    model="claude-3-haiku",  # 사용할 모델
) as stream:
    # 스트림을 통해 생성된 텍스트 출력
    for text in stream.text_stream:
        print(text, end="", flush=True)


In [ ]:
# 교재 속 예시

from langchain_anthropic import ChatAnthropic

# ChatAnthropic 객체 생성
anthropic = ChatAnthropic(model_name="claude-3-5-sonnet-20241022")

# 스트리밍 출력을 위하여 invoke() 대신 stream()을 사용합니다.
answer = anthropic.stream("사랑이 뭔가요?")

# 답변 출력
for chunk in answer:
    print(chunk.content, end="", flush=True)

* *클로드 API 오류로 실습 현재 불가*

---

## **`Perplexity`**

* [링크](https://www.perplexity.ai/): https://www.perplexity.ai/

* **설립연도**: 2022년
* **주요 투자자**: Jeff Bezos, Nvidia, Databricks, Bessemer Venture Partners, IVP, Wayra 등
* **최근 펀딩**: 5억 달러 (2024년 10월)
* **기업 가치**: 약 90억 달러 (2024년 11월 기준)
* **월간 활성 사용자**: 1,500만 명

### **`Perplexity Pro` 정확한 특징**

* **일일 Pro 검색**: 300회
* **AI 모델 선택**: `GPT-5`, `Claude 4 Sonnet`/`Opus`, `Sonar`/`Sonar Pro`, `Grok`[5] 등
* **파일 분석**: `MD`, `PDF`, `CSV`, `이미지 파일` 등 지원
* **가격**: 월 `$20` 또는 연 `$200`

### **`Perplexity API` 사용 방법**

* **가격**

  * ![perplexity-pricing.png](../04_Model/images/perplexity-pricing.png)

  * **API 크레딧 획득**
   * Perplexity Pro 구독 시 **매월 $5 상당의 API 크레딧 제공**

* **API 모델 옵션**
   * Llama 3 기반 모델
   * Perplexity 온라인 LLM
   * 인용 기능 포함

* API 키 발급: [API 콘솔](https://www.perplexity.ai/settings/api)

  * API 키 발급 후 `.env` 파일에 키 저장

    ```bash
    PPLX_API_KEY=이곳에 API 키를 입력하세요.
    ```

* 이후
  
  * `.env`(환경변수)로 불러오기

    ```bash
    import os

    os.environ["PPLX_API_KEY"] = "이곳에 API 키를 입력하세요."
    ```

* **참고**

  * [API 문서](https://docs.perplexity.ai/api-reference/chat-completions)

* `Llama 3.1` 모델 사양

| 모델명                          | 파라미터 수 | 컨텍스트 길이 | 모델 유형             | 주요 특징                                                                 |
|-------------------------------|-------------|---------------|----------------------|--------------------------------------------------------------------------|
| `Meta-Llama-3.1-8B`           | 8B          | 128,000 토큰  | 기본 모델             | 경량화된 모델로 로컬 환경에서의 텍스트 생성에 적합                        |
| `Meta-Llama-3.1-70B`          | 70B         | 128,000 토큰  | 기본 모델             | 기업용 하드웨어에 최적화된 모델로, 텍스트 요약, 분류, 감정 분석 등에 우수한 성능 제공 |
| `Meta-Llama-3.1-405B`         | 405B        | 128,000 토큰  | 기본 모델             | 가장 큰 공개 모델로, 고급 추론, 멀티모달 처리, 대규모 데이터 생성 등에 뛰어난 성능 발휘 |
| `Meta-Llama-3.1-8B-Instruct`  | 8B          | 128,000 토큰  | 지시문 조정 모델       | 사용자 지시문에 따른 응답 생성에 최적화된 모델                            |
| `Meta-Llama-3.1-70B-Instruct` | 70B         | 128,000 토큰  | 지시문 조정 모델       | 사용자 지시문에 따른 응답 생성에 최적화된 모델                            |
| `Meta-Llama-3.1-405B-Instruct`| 405B        | 128,000 토큰  | 지시문 조정 모델       | 사용자 지시문에 따른 응답 생성에 최적화된 모델                            |


  * **컨텍스트 길이 확장** 
    * `Llama 3.1 모델` = 이전 버전인 Llama 3의 8,192 토큰에서 `128,000 토큰`으로 컨텍스트 길이가 대폭 확장 → `더 긴 대화`나 `문서`도 처리

  * **다국어 지원**
    * 영어, 독일어, 프랑스어, 이탈리아어, 포르투갈어, 힌디어, 스페인어, 태국어 등 8개 언어를 지원

  * **모델 유형**
    * `기본 모델`과 `지시문 조정 모델`로 구분
    * `지시문 조정 모델` = **사용자 지시문에 따른 응답 생성에 최적화**

### **`ChatPerplexity` 매개변수**

* `model`  
  * 사용할 언어 모델을 지정 
    * 예시: `llama-3.1-sonar-small-128k-online`
  * 기본 성능과 능력을 결정

* `temperature`  
  * 응답의 무작위성을 조절 (`0.0-1.0`)
  * 0은 결정적, 1은 가장 무작위한 응답 생성

* `top_p`  
  * 토큰 샘플링의 확률 임계값 설정 (`0.0-1.0`)
  * **높을수록 더 다양한 출력 허용**

* `search_domain_filter`  
  * 검색 결과를 지정된 도메인으로 제한
  * `리스트` 형태로 제공 - 예시: [`perplexity.ai`]

* `return_images`: 응답에 `이미지 포함 여부를 결정`하는 불리언 플래그

* `return_related_questions`: `관련 질문 제안 기능`을 `활성화/비활성화`하는 불리언 플래그

* `top_k`
  * 사용할 검색 `결과의 수 제한` 
  * **0은 제한 없음을 의미**

* `streaming`: 응답을 `스트리밍으로 받을지 완성된 형태로 받을지 결정`하는 불리언 플래그

* `presence_penalty`
  * 토큰 반복에 대한 페널티 (`-2.0에서 2.0`)
  * `높을수록 재사용을 억제`

* `frequency_penalty`  
  * `일반적/희귀 토큰 선호도` 조정 (`-2.0에서 2.0`)
  * `높을수록 희귀 토큰 선호`

In [ ]:
# 퍼플렉시티 모델 호출해보기
from perplexipy import PerplexityClient
from langchain_perplexity import ChatPerplexity
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수에서 Perplexity API 키 읽기
api_key = os.getenv("PPLX_API_KEY")

print("\n--- Perplexity API KEY 환경 변수 확인 ---")

if api_key:
    print(f"✅ api_key: {api_key}")
    print(" Perplexity API 키가 환경변수로 잘 설정되었습니다.")
else:
    print("❌ Perplexity API 키가 환경변수로 제대로 설정되지 않았습니다.")
    if not api_key:
        print(" Perplexity_API_KEY 환경변수가 설정되지 않았거나, 비어 있습니다.")

<small>

* 셀 출력

    ```markdown
    --- Perplexity API KEY 환경 변수 확인 ---
    ✅ api_key: pplx-***
    Perplexity API 키가 환경변수로 잘 설정되었습니다.
    ```

In [ ]:
# 환경 변수에 PERPLEXITY_API_KEY 설정 필요
client = PerplexityClient(key=os.getenv("PPLX_API_KEY"))

# 쿼리 수행
result = client.query("Briefly say hello to the world in Swedish.")
print(result)

<small>

* 셀 출력 (2.8s)

    ```markdown
    A brief way to say hello to the world in Swedish is simply **"Hej"** (pronounced like "hey"). It is the most common, friendly, and versatile greeting used in nearly all situations, whether formal or informal[1][2][3][5]. For a slightly more cheerful variant, Swedes sometimes say **"Hej hej!"** which translates to "Hi hi!"[3].
    ```

In [ ]:
# 배치 쿼리
results = client.queryBatch("Different ways to declare a Python list:")
for r in results:
    print(r)

<small>

* 셀 출력 (5.3s)

    ```markdown
    There are several common ways to declare or create a list in Python:

    - **Using square brackets:** You can directly write square brackets `[]` with comma-separated elements inside, e.g., `my_list = [1, 2, 3]`. This is the most straightforward and popular method to create lists, including mixed data types such as integers, strings, or floats[2][3][5].

    - **Using the `list()` constructor:** You can pass any iterable (like a string, tuple, or another list) to the `list()` function to create a new list. For example, `my_list = list((1, 2, 3))` creates a list from a tuple[2][3].

    - **Using list comprehensions:** This is a concise syntax to generate lists by iterating over iterables and optionally applying an expression or condition, e.g., `my_list = [x*2 for x in range(5)]` creates `[0, 2, 4, 6, 8]`[2].

    Additionally, after creating a list, you can add or modify elements using methods like `append()`, `insert()`, or `extend()`[1][4][5].

    Summary table:

    | Method                            | Syntax Example                            | Description                                |
    |----------------------------------|------------------------------------------|--------------------------------------------|
    | Square brackets (List literal)   | `my_list = [1, 'a', 3.14]`                | Directly declare list with elements       |
    | `list()` constructor             | `my_list = list((1, 2, 3))`               | Create list from any iterable              |
    | List comprehension               | `my_list = [x for x in range(5)]`         | Create list using concise iteration syntax |

    These are the primary and most idiomatic ways to declare lists in Python[2][3][5].
    ```

In [ ]:
# 배치 쿼리2
results = client.queryBatch("2024년 노벨문학상 수상자를 조사해 주세요")
for r in results:
    print(r)

<small>    

* 셀 출력 (6.0s)
    
    ```markdown

    2024년 노벨문학상 수상자는 한국의 소설가 **한강**입니다. 한강은 역사적 트라우마에 맞서고 인간 삶의 연약함을 강렬한 시적 산문으로 표현한 작품들로 인정받아 수상하였으며, 이는 한국인 중 최초이자 아시아 여성 작가로서도 최초의 노벨문학상 수상입니다[1][2][3].

    스웨덴 한림원은 한강의 문학을 "역사적 트라우마에 맞서고 인간의 삶의 연약함을 드러내는 강렬한 시적 산문"이라고 평가했습니다[3]. 한강은 2016년 소설 『채식주의자』로 세계 3대 문학상 중 하나인 부커상을 수상한 바 있으며, 이번 수상으로 노벨문학상 역대 121번째 수상자, 18번째 여성 수상자가 되었습니다[2].

    이 수상은 한국 문학과 아시아 문학 모두에 매우 의미 있는 사건이며, 한국인의 노벨상 수상으로는 평화상 수상자인 고 김대중 전 대통령 이후 두 번째 사례입니다[2][4].

    ```

In [ ]:
# 스트리밍 응답
for r in client.queryStreamable("2024년 노벨문학상 수상자를 조사해 주세요"):
    print(r)

<small>

* 셀 출력 (5.4s)

    ```markdown

    202
    4
    년
    노
    벨
    문
    학
    상
    수
    상
    자는
    한국
    의
    소
    설
    가
    **
    한
    강
    **
    입니다
    .
    한
    강
    작
    가는
    아
    시아
    여성
    으로
    는
    최초
    ,
    한국
    인
    으로
    도
    최초
    로
    이
    권
    위
    있는
    상
    을
    받
    았습니다
    [1]
    [2]
    [4].


    스
    웨
    덴
    한
    림
    원
    은
    한
    강
    작
    가
    에게
    상
    을
    수
    여
    한
    이유
    로
    “
    역
    사
    적
    트
    라우
    마
    에
    맞
    서
    고
    인간
    삶
    의
    연
    약
    함
    을
    드
    러
    내
    는
    강
    렬
    한
    시
    적
    산
    문
    ”
    을
    썼
    다는
    점
    을
    꼽
    았습니다
    .
    그녀
    의
    작품
    은
    몸
    과
    영
    혼
    ,
    산
    자
    와
    죽
    은
    자
    사이
    의
    연결
    에
    대한
    독
    특
    한
    인
    식을
    담
    고
    있으며
    ,
    시
    적
    이고
    실
    험
    적인
    스타일
    로
    현대
    산
    문의
    혁
    신
    을
    이
    끌
    었다
    고
    평가
    받
    았습니다
    [2]
    [4].


    한
    강
    은
    
    53
    세
    의
    비교
    적
    젊
    은
    나
    이에
    수
    상
    했
    으며
    ,
    이는
    노
    벨
    문
    학
    상
    수
    상
    자
    평균
    연
    령
    들
    에
    비
    해
    이
    례
    적
    입니다
    .
    국내
    외
    에서
    부
    커
    상
    등
    여러
    국제
    문
    학
    상을
    받은
    경
    력
    과
    더
    불
    어
    이번
    노
    벨
    문
    학
    상
    수
    상
    은
    한국
    문
    학
    과
    문화
    의
    세계
    적
    위
    상을
    높
    이는
    큰
    성
    과
    로
    여
    겨
    집
    니다
    [1]
    [4].


    한
    강
    작
    가
    의
    대표
    작
    으로
    는
    『
    채
    식
    주의
    자
    』
    가
    널
    리
    알려
    져
    있으며
    ,
    이
    작품
    을
    통해
    전
    세계
    독
    자
    들에게
    큰
    반
    향
    을
    일
    으
    켰
    습니다
    [3].
    


    요
    약
    하면
    ,
    
    202
    4
    년
    노
    벨
    문
    학
    상
    은
    한국
    의
    한
    강
    작
    가
    가
    수
    상
    했
    으며
    ,
    그녀
    는
    역사
    적
    트
    라우
    마
    와
    인간
    의
    연
    약
    함
    을
    주
    제로
    한
    시
    적
    산
    문
    으로
    큰
    평가
    를
    받
    았습니다
    .
    이는
    한국
    문
    학
    사
    에서
    매우
    중요한
    이
    정
    표
    가
    되
    었습니다
    [1]
    [2]
    [4].

    ```

In [ ]:
# 사용할 수 있는 모델 리스트 조회
print(client.models)

<small>

* 셀 출력

    ```python
    OrderedDict({'sonar-reasoning-pro': ModelInfo(parameterCount='8B', contextLength=127072, modelType='Sonar', availability='Perplexity'), 'sonar-reasoning': ModelInfo(parameterCount='8B', contextLength=127072, modelType='Sonar', availability='Perplexity'), 'sonar-pro': ModelInfo(parameterCount='8B', contextLength=127072, modelType='Sonar', availability='Perplexity'), 'sonar': ModelInfo(parameterCount='8B', contextLength=127072, modelType='Sonar', availability='Perplexity'), 'r1-1776': ModelInfo(parameterCount='8B', contextLength=127072, modelType='Sonar', availability='Perplexity')})
    ```

* 교재에 있는 모델과 다름을 확인

  * `sonar-reasoning-pro`

  * `sonar-reasoning`

  * `sonar-pro`

  * `sonar`

  * `r1-1776`

In [ ]:
# 사용할 수 있는 모델 중 sonar로 시도
# 이전 시도에서 에러가 났던 매개변수 제거

from langchain_perplexity import ChatPerplexity
import os

# API 키 환경 변수에서 불러오기
api_key = os.getenv("PPLX_API_KEY")

# 사용할 모델 중 하나 선택 (예: sonar-pro)
perplexity = ChatPerplexity(
    model="sonar-pro",                      # 사용할 모델 지정
    temperature=0.7,                        # 텍스트 다양성 조정
    top_p=0.9,                              # 샘플링 파라미터
    streaming=False                         # 스트리밍 여부 설정
)

# 메시지 입력 예시
messages = [
    {"role": "user", "content": "2024년 노벨문학상 수상자를 조사해 주세요"}
]

# Perplexity API 호출
response = perplexity.invoke(messages)

# 응답 내용 출력
print(response.content)

# 응답에서 인용된 출처들 출력
print("\n--- 인용된 출처 ---")
for i, citation in enumerate(response.citations):
    print(f"[{i+1}] {citation}")

<small>

* 셀 출력 (5.2s)

    ```markdown
    WARNING! top_p is not a default parameter.
                        top_p was transferred to model_kwargs.
                        Please confirm that top_p is what you intended.
    2024년 노벨문학상 수상자는 **한국의 소설가 한강**입니다[1][2][3][4][5].

    한강은 한국인 최초로 노벨문학상을 수상했으며, 아시아 여성 작가로서도 최초의 수상자입니다[2][5]. 스웨덴 한림원은 2024년 10월 10일(현지 시각) 한강의 수상 이유로 "**역사적 트라우마에 맞서고 인간 삶의 연약함을 폭로하는 강렬한 시적 산문**"을 꼽았습니다[2][3][4]. 한강은 이미 2016년 소설 『채식주의자』로 맨부커상(Man Booker International Prize)을 수상하며 세계적으로 주목받은 바 있습니다[2].

    이번 수상은 2000년 고(故) 김대중 대통령의 노벨평화상 이후 24년 만에 한국인으로서는 두 번째 노벨상 수상이기도 합니다[4][5]. 한강은 121번째 노벨문학상 수상자이자, 18번째 여성 수상자입니다[2][4].

    --- 인용된 출처 ---
    ```

* 오류 메시지

    ```python
    ---------------------------------------------------------------------------
    AttributeError                            Traceback (most recent call last)
    Cell In[63], line 31
        29 # 응답에서 인용된 출처들 출력
        30 print("\n--- 인용된 출처 ---")
    ---> 31 for i, citation in enumerate(response.citations):
        32     print(f"[{i+1}] {citation}")

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/pydantic/main.py:991, in BaseModel.__getattr__(self, item)
        988     return super().__getattribute__(item)  # Raises AttributeError if appropriate
        989 else:
        990     # this is the current error
    --> 991     raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')

    AttributeError: 'AIMessage' object has no attribute 'citations'

    ```

* [참고 가이드](https://docs.perplexity.ai/guides/chat-completions-guide?utm_source=chatgpt.com)

In [ ]:
# 사용할 수 있는 모델 중 sonar로 시도
# 이전 시도에서 에러가 났던 매개변수 제거

from langchain_perplexity import ChatPerplexity
import os
import re                                   # re 모듈 임포트해서 정규 표현식 기능 사용 가능하도록 함

# API 키 환경 변수에서 불러오기
api_key = os.getenv("PPLX_API_KEY")

# 사용할 모델 중 하나 선택 (예: sonar-pro)
perplexity = ChatPerplexity(
    model="sonar-pro",                      # 사용할 모델 지정
    temperature=0.7,                        # 텍스트 다양성 조정
    #top_p=0.9,                             # 주석 처리 (오류 방지)
    streaming=False                         # 스트리밍 여부 설정
)

# 메시지 입력 예시
messages = [
    {"role": "user", "content": "2024년 노벨문학상 수상자를 조사해 주세요"}
]

# Perplexity API 호출
response = perplexity.invoke(messages)

# 응답 내용 출력
print(response.content)

# 응답에서 인용된 출처들 추출 (정규 표현식을 사용하여 [1][2][3] 등의 형식 추출)
citations = re.findall(r'\[(\d+)\]', response.content)

# 인용된 출처 출력
print("\n--- 인용된 출처 ---")
for i, citation in enumerate(citations):
    print(f"[{i+1}] {citation}")

<small>

* 셀 출력 (8.9s)

    ```markdown
    2024년 노벨문학상은 **한국의 소설가 한강**이 수상했습니다[1][2][3][4].

    한강은 2024년 10월 스웨덴 한림원으로부터 “역사적 트라우마에 맞서고 인간 삶의 연약함을 폭로하는 강렬한 시적 산문”을 쓴 작가로 선정되었으며, 한국인으로는 최초의 노벨문학상 수상자입니다[2][3][4]. 한강은 2016년 『채식주의자』로 맨부커 인터내셔널상을 수상한 경력이 있으며, 이번 수상은 2000년 김대중 전 대통령의 노벨평화상 이후 두 번째 한국인의 노벨상 수상 기록입니다[2][3][4]. 

    또한 한강은 아시아 여성 작가로는 최초로 노벨문학상을 수상했으며, 전체 역대 121번째, 여성으로는 18번째 수상자입니다[2][3][4]. 스웨덴 한림원은 한강의 문학 세계를 “몸과 영혼, 산 자와 죽은 자 사이의 연결에 대한 독특한 인식”과 “시적이고 실험적인 스타일로 현대 산문의 혁신가”라고 평가했습니다[3].

    --- 인용된 출처 ---
    [1] 1
    [2] 2
    [3] 3
    [4] 4
    [5] 2
    [6] 3
    [7] 4
    [8] 2
    [9] 3
    [10] 4
    [11] 2
    [12] 3
    [13] 4
    [14] 3
    ```

---

* *next: **`Cohere`***